In [3]:
import os
import nxsdk.api.n2a as nx
import numpy as np
import matplotlib.pyplot as plt
from nxsdk.graph.monitor.probes import *
from nxsdk.graph.processes.phase_enums import Phase

In [4]:
import matplotlib.pyplot as plt

In [5]:
def create_network(numArms, neuronsPerArm, epochs, weights, probabilities):
    assert len(weights) == numArms, "Must provide number of weights equal to number of arms."
    assert len(probabilities) == numArms, "Must provide probability for each arm's reward."
    for p in probabilities:
        assert p in range(0,100), "Probabilities must be represented as int from 0-100."
        
    totalNeurons = neuronsPerArm * numArms

    #create the network
    net = nx.NxNet()
    #set up the noisy source neuron prototype
    p_2Fire = nx.CompartmentPrototype(biasMant=0,
                            biasExp=0,
                            vThMant=4,
                            compartmentVoltageDecay=0,
                            compartmentCurrentDecay=0,

                            enableNoise=1,
                            randomizeVoltage=1,
                            randomizeCurrent=0,
                            noiseMantAtCompartment=2,
                            noiseExpAtCompartment=7,
                            functionalState=nx.COMPARTMENT_FUNCTIONAL_STATE.IDLE,
                             logicalCoreId=1)
    #set up the integrative prototype
    p_Compare = nx.CompartmentPrototype(biasMant=0,
                            biasExp=0,
                            vThMant=100,
                            compartmentVoltageDecay=0,
                            compartmentCurrentDecay=2048,

                            enableNoise=1,
                            randomizeVoltage=0,
                            randomizeCurrent=1,
                            noiseMantAtCompartment=0,
                            noiseExpAtCompartment=7,
                            functionalState=nx.COMPARTMENT_FUNCTIONAL_STATE.IDLE,
                             logicalCoreId=0)

    #set up the learning rule that will control weight changes
    lr = net.createLearningRule(dw='u0*r1',
                                r1Impulse=1,
                                r1TimeConstant=1,
                                tEpoch=32)
                                #TODO - can inc/dec weight based on reward

    #create the connections which drive the integrator
    exh_connx = nx.ConnectionPrototype(weight=2,
                                    delay=0,
                                    enableLearning=1,
                                    learningRule=lr,
                                    signMode=nx.SYNAPSE_SIGN_MODE.MIXED)
    inh_connx = nx.ConnectionPrototype(weight=2,
                                    delay=0,
                                    signMode=nx.SYNAPSE_SIGN_MODE.MIXED)





    #create inhibitory, excitatory, and integrative compartment groups
    inhDriver = net.createCompartmentGroup(size=totalNeurons, prototype=p_2Fire)
    exhDriver = net.createCompartmentGroup(size=totalNeurons, prototype=p_2Fire)
    comparator = net.createCompartmentGroup(size=totalNeurons, prototype=p_Compare)

    exhGrp = exhDriver.connect(comparator,
                  prototype=exh_connx,
                  weight= np.repeat(weights, neuronsPerArm) * np.identity(totalNeurons),
                 connectionMask=np.identity(totalNeurons))

    inhGrp = inhDriver.connect(comparator,
                  prototype=inh_connx,
                  weight=-50*np.identity(totalNeurons),
                 connectionMask=np.identity(totalNeurons))

    #setup the spike monitor
    customSpikeProbeCond = SpikeProbeCondition(tStart=10000000)
    spikes = comparator.probe(nx.ProbeParameter.SPIKE, customSpikeProbeCond)

    #compile the network so we can add channels for the SNIPs
    compiler = nx.N2Compiler()
    board = compiler.compile(net)

    #setup the management SNIP to calculate rewards and choose numArms
    includeDir = os.getcwd()
    learning = board.createSnip(Phase.EMBEDDED_MGMT,
                                 includeDir=includeDir,
                                 cFilePath = includeDir + "/management.c",
                                 funcName = "run_cycle",
                                 guardName = "check")

    #create a channel to communicate with the Lakemont what the probability of each arm is
    #have to transfer the number of arms, neurons per arm, and the probeID listening to each neuron
    setupChannel = board.createChannel(b'setupChannel', "int", numArms + 2)
    setupChannel.connect(None, learning)
    dataChannel = board.createChannel(b'dataChannel', "int", (epochs+1) * numArms * neuronsPerArm)
    dataChannel.connect(learning, None)

    board.startDriver()
    #first send the # of arms
    setupChannel.write(1, [numArms])
    #the # of neurons per arm
    setupChannel.write(1, [neuronsPerArm])
    #and the probeID for each neuron 
    setupChannel.write(numArms, probabilities)

    return (board, spikes, dataChannel)

In [6]:
probs = [90, 10, 10, 10, 10]
weights = np.linspace(50, 100, num=5, dtype='int')
numArms = 5
neuronsPerArm = 5
epochs = 10
(board, mons, dchannel) = create_network(numArms, neuronsPerArm, epochs, weights, probs)
    

INFO:DRV:  SLURM is being run in background


/homes/wolin/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:99: NxDeprecationWarning: startDriver is being deprecated. Use start instead.


INFO:DRV:  Connecting to 10.212.98.103:41783
INFO:DRV:      Host server up..............Done 0.24s
INFO:DRV:      Compiling Embedded snips....Done 0.29s
INFO:DRV:      Encoding axons/synapses.....Done 9.80ms
INFO:DRV:      Encoding probes.............Done 0.84ms
INFO:HST:  Args chip=0 cpu=0 /homes/wolin/python3_venv/lib/python3.5/site-packages/nxsdk/driver/compilers/../../../temp/1573180087.848707/launcher_chip0_lmt0.bin --chips=1 --remote-relay=1 --epoch=32 
INFO:DRV:      Booting up..................Done 2.64s
INFO:HST:  Lakemont_driver...


In [7]:
board.run(100*epochs+1, aSync=True)

INFO:DRV:      Transferring probes.........Done 4.10ms
INFO:DRV:      Transferring spikes.........Done 0.80ms
INFO:DRV:      Configuring registers.......Done 9.89ms


In [8]:
counts =  dchannel.read(epochs*numArms*neuronsPerArm)

In [9]:
board.disconnect()

INFO:HST:  chip=0 cpu=0 halted, status=0x0


In [10]:
len(counts)

250

In [11]:
rsc = np.reshape(counts, (epochs, numArms, neuronsPerArm))

In [12]:
rsc

array([[[ 3,  4,  4,  3,  3],
        [ 4,  6, 10,  2,  3],
        [ 6,  0,  0,  0,  0],
        [ 2,  1,  1,  2,  1],
        [ 0,  3,  4,  4,  0]],

       [[ 7,  7,  7,  9,  4],
        [ 3,  8,  7,  8,  8],
        [ 7,  0,  0,  0,  0],
        [ 1,  2,  0,  2,  2],
        [ 0,  6,  2,  2,  5]],

       [[ 3,  5,  4,  5,  7],
        [ 7, 11,  8, 10,  8],
        [ 9,  0,  0,  1,  0],
        [ 0,  4,  2,  1,  3],
        [ 0,  3,  4,  4,  2]],

       [[ 7,  6,  5,  7,  6],
        [ 6,  8,  9,  8,  4],
        [ 5,  0,  0,  0,  0],
        [ 0,  0,  2,  3,  0],
        [ 2,  3,  5,  1,  4]],

       [[ 4,  6,  8,  6,  2],
        [ 3,  6,  9,  3,  6],
        [ 8,  0,  0,  2,  0],
        [ 0,  3,  2,  2,  0],
        [ 3,  7,  3,  0,  6]],

       [[ 6,  5,  8,  7,  1],
        [ 3,  8, 10,  3,  8],
        [ 7,  0,  0,  0,  0],
        [ 0,  1,  1,  1,  1],
        [ 4,  3,  1,  7,  6]],

       [[ 4,  8,  3,  9,  5],
        [ 6,  7,  5,  7,  2],
        [ 3,  0,  0,  0,  0]

In [19]:
len(mons[0].probes)

25

In [21]:
[p.n2Probe.counterId for p in mons[0].probes]

[43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42]

doesn't fix things being out of order

In [ ]:
pids = np.array()

In [13]:
pids = np.array([[mons[j][0].probes[i].n2Probe.counterId for i in range(5)] for j in range(5)])

AttributeError: 'NxProbe' object has no attribute 'probes'

In [ ]:
pids

In [41]:
pids_offset = pids - 32

In [45]:
pids_offset

array([[ 4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18],
       [19, 20, 21, 22, 23],
       [ 0,  1,  2,  3, 24]])

In [43]:
pids_offset[0,:]

array([4, 5, 6, 7, 8])

In [47]:
rsc[0,:]

array([[7, 7, 3, 4, 0],
       [0, 0, 1, 0, 1],
       [3, 3, 0, 0, 4],
       [3, 2, 1, 4, 3],
       [6, 4, 6, 4, 6]])

Get the firing counts by group and see if they make sense

In [53]:
fires = [np.array([np.ravel(rsc[i, :])[pids_offset[j,:]] for i in range(epochs)]) for j in range(numArms)]

In [54]:
len(fires)

5

In [55]:
[f.shape for f in fires]

[(10, 5), (10, 5), (10, 5), (10, 5), (10, 5)]

In [56]:
[np.sum(f) for f in fires]

[8, 77, 178, 243, 353]

In [56]:
np.sort(np.unique([[mons[j][0].probes[i].n2Probe.counterId for i in range(5)] for j in range(5)]))

array([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50, 51, 52, 53, 54, 55, 56])

In [36]:
mons[0][0].data

array([], shape=(5, 0), dtype=int64)

In [37]:
[np.sum(mons[i][0].data,axis=1) for i in range(5)]

[array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0])]

In [24]:
[np.sum(mons[i][0].data,axis=1) for i in range(5)]

[array([1, 6, 4, 1, 3]),
 array([140, 139, 136, 135, 144]),
 array([136, 131, 149, 130, 144]),
 array([142, 147, 137, 150, 143]),
 array([152, 139, 144, 138, 146])]

In [23]:
[np.sum(mons[i][0].data) for i in range(5)]

[15, 694, 690, 719, 719]